In [8]:
import pandas as pd
import requests

In [27]:
register = pd.read_csv("../government-organisation.csv").sort_values("index-entry-number")
register

,index-entry-number,entry-number,entry-timestamp,key,government-organisation,name,website,start-date,end-date
988,1,1,2017-04-25T12:59:14Z,D1,D1,Attorney General's Office,https://www.gov.uk/government/organisations/at...,NaN,NaN
987,2,2,2017-04-25T12:59:14Z,D2,D2,Cabinet Office,https://www.gov.uk/government/organisations/ca...,NaN,NaN
986,3,3,2017-04-25T12:59:14Z,D3,D3,"Department for Business, Innovation & Skills",https://www.gov.uk/government/organisations/de...,NaN,2016-07-14
985,6,6,2017-04-25T12:59:14Z,D6,D6,Department for Education,https://www.gov.uk/government/organisations/de...,NaN,NaN
984,7,7,2017-04-25T12:59:14Z,D7,D7,"Department for Environment, Food & Rural Affairs",https://www.gov.uk/government/organisations/de...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4,1009,1009,2019-01-21T16:23:43Z,OT1266,OT1266,Single Financial Guidance Body,https://www.gov.uk/government/organisations/si...,NaN,NaN
3,1010,1010,2019-01-21T16:31:46Z,OT1267,OT1267,Employment Agency Standards Inspectorate,https://www.gov.uk/government/organisations/em...,NaN,NaN
2,1011,1011,2019-01-21T16:44:12Z,OT1268,OT1268,UK Council for Internet Safety,https://www.gov.uk/government/organisations/uk...,NaN,NaN
1,1013,1013,2020-09-01T17:55:19Z,EA61,EA61,FCO Services,https://www.gov.uk/government/organisations/fc...,NaN,NaN


In [5]:
SOURCE_URL = "https://www.gov.uk/api/content/government/organisations"

In [12]:
r = requests.get(SOURCE_URL)
r.json()

new_organisations = []
for orgtype, orgs in r.json().get("details", {}).items():
    for org in orgs:
        new_organisations.append({
            "orgtype": orgtype,
            **org
        })

new_orgs = pd.json_normalize(new_organisations)

In [23]:
register_new = pd.DataFrame({
    "index-entry-number": None,
    "entry-number": None,
    "entry-timestamp": None,
    "key": new_orgs["analytics_identifier"],
    "government-organisation": new_orgs["analytics_identifier"],
    "name": new_orgs["title"],
    "website": new_orgs["href"].apply(lambda x: "https://www.gov.uk" + x if x else None),
    "start-date": None,
    "end-date": None,
}, index=new_orgs.index)
register_new

,index-entry-number,entry-number,entry-timestamp,key,government-organisation,name,website,start-date,end-date
0,None,None,None,OT532,OT532,"Prime Minister's Office, 10 Downing Street",https://www.gov.uk/government/organisations/pr...,None,None
1,None,None,None,OT564,OT564,Bona Vacantia,https://www.gov.uk/government/organisations/bo...,None,None
2,None,None,None,OT535,OT535,Border Force,https://www.gov.uk/government/organisations/bo...,None,None
3,None,None,None,OT1051,OT1051,British Cattle Movement Service,https://www.gov.uk/government/organisations/br...,None,None
4,None,None,None,OT1201,OT1201,Cabinet Office Board,https://www.gov.uk/government/organisations/ca...,None,None
...,...,...,...,...,...,...,...,...,...
580,None,None,None,OT316,OT316,Westminster Foundation for Democracy,https://www.gov.uk/government/organisations/we...,None,None
581,None,None,None,EA62,EA62,Wilton Park,https://www.gov.uk/government/organisations/wi...,None,None
582,None,None,None,OT547,OT547,Yorkshire Dales National Park Authority,https://www.gov.uk/government/organisations/yo...,None,None
583,None,None,None,EA697,EA697,Youth Justice Agency of Northern Ireland,https://www.gov.uk/government/organisations/yo...,None,None


In [49]:
register_to_add = register_new[~register_new["key"].isin(register["key"])]
register_to_add.loc[:, "index-entry-number"] = register_merged["index-entry-number"].max() + 1 + register_to_add.reset_index().index
register_to_add.loc[:, "entry-number"] = register_to_add["index-entry-number"]
register_to_add.loc[:, "entry-timestamp"] = pd.Timestamp.now()
register_to_add

register_merged = pd.concat([
    register,
    register_to_add,
], ignore_index=True).sort_values("index-entry-number", ascending=False)
register_merged

C:\Users\drkan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
C:\Users\drkan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\drkan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning:

,index-entry-number,entry-number,entry-timestamp,key,government-organisation,name,website,start-date,end-date
1091,1323,1323,2022-01-17 15:35:19.612520,PB1221,PB1221,UK Research and Innovation,https://www.gov.uk/government/organisations/uk...,None,None
1090,1322,1322,2022-01-17 15:35:19.612520,EA1328,EA1328,UK Health Security Agency,https://www.gov.uk/government/organisations/uk...,None,None
1089,1321,1321,2022-01-17 15:35:19.612520,OT1288,OT1288,UK Asset Resolution Limited,https://www.gov.uk/government/organisations/uk...,None,None
1088,1320,1320,2022-01-17 15:35:19.612520,PB1327,PB1327,Trade Remedies Authority,https://www.gov.uk/government/organisations/tr...,None,None
1087,1319,1319,2022-01-17 15:35:19.612520,OT1314,OT1314,Submarine Delivery Agency,https://www.gov.uk/government/organisations/su...,None,None
...,...,...,...,...,...,...,...,...,...
4,7,7,2017-04-25T12:59:14Z,D7,D7,"Department for Environment, Food & Rural Affairs",https://www.gov.uk/government/organisations/de...,NaN,NaN
3,6,6,2017-04-25T12:59:14Z,D6,D6,Department for Education,https://www.gov.uk/government/organisations/de...,NaN,NaN
2,3,3,2017-04-25T12:59:14Z,D3,D3,"Department for Business, Innovation & Skills",https://www.gov.uk/government/organisations/de...,NaN,2016-07-14
1,2,2,2017-04-25T12:59:14Z,D2,D2,Cabinet Office,https://www.gov.uk/government/organisations/ca...,NaN,NaN


In [50]:
register_merged.to_csv("../government-organisation.csv", index=False)

In [33]:
register_merged["index-entry-number"].max()

1014

,index-entry-number,entry-number,entry-timestamp,key,government-organisation,name,website,start-date,end-date
5,1015,1015,2022-01-17 15:34:33.252215,OT1329,OT1329,Central Digital and Data Office,https://www.gov.uk/government/organisations/ce...,None,None
6,1016,1016,2022-01-17 15:34:33.252215,OT1223,OT1223,Centre for Connected and Autonomous Vehicles,https://www.gov.uk/government/organisations/ce...,None,None
7,1017,1017,2022-01-17 15:34:33.252215,OT1280,OT1280,Centre for Data Ethics and Innovation,https://www.gov.uk/government/organisations/ce...,None,None
12,1018,1018,2022-01-17 15:34:33.252215,OT1281,OT1281,Civil Service Group,https://www.gov.uk/government/organisations/ci...,None,None
13,1019,1019,2022-01-17 15:34:33.252215,OT1304,OT1304,Civil Service HR,https://www.gov.uk/government/organisations/ci...,None,None
...,...,...,...,...,...,...,...,...,...
542,1113,1113,2022-01-17 15:34:33.252215,OT1314,OT1314,Submarine Delivery Agency,https://www.gov.uk/government/organisations/su...,None,None
548,1114,1114,2022-01-17 15:34:33.252215,PB1327,PB1327,Trade Remedies Authority,https://www.gov.uk/government/organisations/tr...,None,None
555,1115,1115,2022-01-17 15:34:33.252215,OT1288,OT1288,UK Asset Resolution Limited,https://www.gov.uk/government/organisations/uk...,None,None
560,1116,1116,2022-01-17 15:34:33.252215,EA1328,EA1328,UK Health Security Agency,https://www.gov.uk/government/organisations/uk...,None,None


In [42]:
register_new[~register_new["key"].isin(register["key"])].reset_index().index

RangeIndex(start=0, stop=103, step=1)